In [64]:
print('Hello World')

Hello World


In [1]:
import pandas as pd
import os
import datetime as dt
os.listdir()


['.ipynb_checkpoints',
 'AnlagenEegSolar_37.xml',
 'BESCHREIBUNG_obsgermany_climate_daily_solar_de.pdf',
 'EinheitenSolar_38.xml',
 'Links.txt',
 'solarEnergyFeedIn_BY_2023-01-01_2023-12-31.csv',
 'StackFuel_Project.ipynb',
 'Station_03668.txt',
 'Station_05792.txt',
 'Stromerzeuger_2024-01-01.csv',
 'tageswerte_ST_03668_row.zip',
 'tageswerte_ST_05792_row.zip']

In [60]:
# data source: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/solar/

df = pd.DataFrame({'date': pd.date_range(start='1/1/2023', freq='1d', periods=365)})

stationsid = ['03668','05792']
for id in stationsid:
    stationsname = 'Station_'+id+'.txt'
    df_stat = pd.read_csv(stationsname,sep=';',parse_dates=['MESS_DATUM'],usecols=['MESS_DATUM','FD_STRAHL','FG_STRAHL','SD_STRAHL'])
    mask = (df_stat.loc[:,'MESS_DATUM'] >= '2023-01-01') & (df_stat.loc[:,'MESS_DATUM'] <= '2023-12-31')
    df_stat.columns =['date',id+'_FD_STRAHL',id+'_FG_STRAHL',id+'_SD_STRAHL']
    df = df.merge(df_stat.loc[mask,:],on='date')

df.head()
# for-loop für Stationen, Umbenennung der Columns in z.B. FD_StatID

,date,03668_FD_STRAHL,03668_FG_STRAHL,03668_SD_STRAHL,05792_FD_STRAHL,05792_FG_STRAHL,05792_SD_STRAHL
0,2023-01-01,253.0,308.0,2.3,181.0,662.0,7.8
1,2023-01-02,251.0,357.0,3.2,315.0,581.0,6.4
2,2023-01-03,213.0,394.0,4.3,262.0,263.0,0.0
3,2023-01-04,126.0,134.0,0.0,267.0,396.0,3.0
4,2023-01-05,250.0,336.0,1.7,204.0,229.0,0.0


In [61]:
# data source: https://netztransparenz.tennet.eu/de/strommarkt/transparenz/transparenz-deutschland/netzkennzahlen/tatsaechliche-und-prognostizierte-solarenergieeinspeisung/bayern/
# filtered for 2023-01-01 to 2023-12-31

df_en_raw = pd.read_table('solarEnergyFeedIn_BY_2023-01-01_2023-12-31.csv',sep=';',parse_dates=['Datum'],decimal=',')

df_en = pd.DataFrame({'date': pd.date_range(start='1/1/2023', freq='1d', periods=365)})
df_en.loc[:,'Prog_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Prognostiziert in MW'].sum()).iloc[:,1]
df_en.loc[:,'Act_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Tatsaechlich in MW'].sum()).iloc[:,1]
df = df.merge(df_en,on='date')
df.head()

,date,03668_FD_STRAHL,03668_FG_STRAHL,03668_SD_STRAHL,05792_FD_STRAHL,05792_FG_STRAHL,05792_SD_STRAHL,Prog_in_MW,Act_in_MW
0,2023-01-01,253.0,308.0,2.3,181.0,662.0,7.8,59750.8,66402.6
1,2023-01-02,251.0,357.0,3.2,315.0,581.0,6.4,54349.6,57194.2
2,2023-01-03,213.0,394.0,4.3,262.0,263.0,0.0,30626.6,36114.8
3,2023-01-04,126.0,134.0,0.0,267.0,396.0,3.0,14734.7,21821.3
4,2023-01-05,250.0,336.0,1.7,204.0,229.0,0.0,22221.0,21564.3


In [62]:
#file = open("Einheitensolar/EinheitenSolar_38.xml", "r")
#print(file.read()[:3500])

In [63]:
# data source: https://download.marktstammdatenregister.de/Gesamtdatenexport_20240306_23.2.zip
# EinheitenSolar_1-39.xml

import xml.etree.ElementTree as ET

files = list(range(1,40))
df_mod_raw = pd.DataFrame(columns=['Datum', 'PLZ', 'Bruttoleistung','Nettonennleistung','Inbetriebnahme'])
counter = 0
for file in files:
    source = 'Einheitensolar/EinheitenSolar_'+str(file)+'.xml'
    tree = ET.parse(source)
    root = tree.getroot()
    
    for einheit in root.findall('EinheitSolar'):
        try: 
            land = einheit.find('Bundesland').text
        except:
            land = 'Unbekannt'
        if land == '1403': # Bundesland 1403 = Bayern
            try: 
                inbetrieb = einheit.find('Inbetriebnahmedatum').text
            except:
                inbetrieb = 'Ausser_Betrieb'
            
            if inbetrieb.startswith('2023'):  
                plz = einheit.find('Postleitzahl').text 
                brutto = float(einheit.find('Bruttoleistung').text)
                netto = float(einheit.find('Nettonennleistung').text)
                datum = einheit.find('DatumLetzteAktualisierung').text
        
                datum = datum[:10]
                df_mod_raw.loc[counter] = [datum,plz,brutto,netto,inbetrieb]
                counter = counter + 1





In [65]:
df_mod = pd.DataFrame({'date': pd.date_range(start='1/1/2023', freq='1d', periods=365)})
df_mod.loc[:,'Bruttoleistung'] = pd.DataFrame(df_mod_raw.groupby(['Inbetriebnahme'], as_index=False)['Bruttoleistung'].sum()).iloc[:,1]
df_mod.loc[:,'Nettonennleistung'] = pd.DataFrame(df_mod_raw.groupby(['Inbetriebnahme'], as_index=False)['Nettonennleistung'].sum()).iloc[:,1]
df_mod.loc[:,'Bruttoleistung_kumulativ'] = df_mod.loc[:,'Bruttoleistung'].cumsum()+18643291
df_mod.loc[:,'Nettonennleistung_kumulativ'] = df_mod.loc[:,'Nettonennleistung'].cumsum()+17044680
df = df.merge(df_mod,on='date')
# netto vor 2023: 17.044.680 kW
# brutto vor 2023: 18.643.291 kW

,date,Bruttoleistung,Nettonennleistung,Bruttoleistung_kumulativ,Nettonennleistung_kumulativ
0,2023-01-01,7638.064,7052.714,1.865093e+07,1.705173e+07
1,2023-01-02,21244.711,19219.613,1.867217e+07,1.707095e+07
2,2023-01-03,6311.835,5687.815,1.867849e+07,1.707664e+07
3,2023-01-04,6880.479,6221.365,1.868537e+07,1.708286e+07
4,2023-01-05,6440.564,5826.238,1.869181e+07,1.708869e+07


In [67]:
df.to_csv('df_2023.csv')

In [69]:
print(df.shape)
df

(365, 13)


,date,03668_FD_STRAHL,03668_FG_STRAHL,03668_SD_STRAHL,05792_FD_STRAHL,05792_FG_STRAHL,05792_SD_STRAHL,Prog_in_MW,Act_in_MW,Bruttoleistung,Nettonennleistung,Bruttoleistung_kumulativ,Nettonennleistung_kumulativ
0,2023-01-01,253.0,308.0,2.3,181.0,662.0,7.8,59750.80,66402.6,7638.064,7052.714,1.865093e+07,1.705173e+07
1,2023-01-02,251.0,357.0,3.2,315.0,581.0,6.4,54349.60,57194.2,21244.711,19219.613,1.867217e+07,1.707095e+07
2,2023-01-03,213.0,394.0,4.3,262.0,263.0,0.0,30626.60,36114.8,6311.835,5687.815,1.867849e+07,1.707664e+07
3,2023-01-04,126.0,134.0,0.0,267.0,396.0,3.0,14734.70,21821.3,6880.479,6221.365,1.868537e+07,1.708286e+07
4,2023-01-05,250.0,336.0,1.7,204.0,229.0,0.0,22221.00,21564.3,6440.564,5826.238,1.869181e+07,1.708869e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,265.0,349.0,2.2,164.0,637.0,7.7,49308.82,57761.5,3113.920,2749.650,2.241328e+07,2.037055e+07
361,2023-12-28,115.0,115.0,0.0,335.0,449.0,4.0,36315.35,31177.6,1968.520,1690.195,2.241525e+07,2.037225e+07
362,2023-12-29,183.0,190.0,0.3,407.0,565.0,5.9,36246.40,32275.7,13647.759,11110.429,2.242890e+07,2.038336e+07
363,2023-12-30,270.0,311.0,1.9,135.0,682.0,8.0,47332.25,49406.9,1634.305,1307.468,2.243053e+07,2.038466e+07
